# Interpreting DICE2016R2 Results

This can only be run on the 2016r2 branch of DICE at the moment

In [ ]:
using DICE
using FileIO
using DataFrames

This file is not in the repo - only for testing currently.

In [ ]:
base = load("R2BaseResults.jld2");

In [1]:
quint_ψ₂ = [0.00061, 0.00141, 0.00227, 0.00242, 0.00464]; #Coefficients for the damage equation kkdam
quint_gₐ₀ = [-0.00241, 0.04307, 0.076, 0.09843, 0.16508]; #Coefficients for the PROD equation kkprod
quint_t2xco2 = [2.00767, 2.51586, 3.1, 3.38802, 4.49126]; #Coefficients for the climate equation kkets
quint_gσ₀ = [-0.02002, -0.01689, -0.0152, -0.01334, -0.01055]; #Coefficients for emissions intensity kksig
quint_mueq = [233.59, 292.95, 360, 394.35, 519.33]; #Coefficients for the carbon cycle kkcarb

mappedphi = Array{Float64,1}();
mappedt2 = Array{Float64,1}();
mappedga = Array{Float64,1}();
mappedmueq = Array{Float64,1}();
mappedgs = Array{Float64,1}();
for phi in 1 #quint_ψ₂
    for t2 in 1 #quint_t2xco2
        for ga in 1 #quint_gₐ₀
            for mu in 1 #quint_mueq
                for gs in 1:5 #quint_gσ₀
                    push!(mappedphi, quint_ψ₂[phi]);
                    push!(mappedt2, quint_t2xco2[t2]);
                    push!(mappedga, quint_gₐ₀[ga]);
                    push!(mappedmueq, quint_mueq[mu]);
                    push!(mappedgs, quint_gσ₀[gs]);
                end
            end
        end
    end
end


In [ ]:
#1 -> 2015, 2 -> 2020, 8 -> 2050, 18 -> 2100, 38 -> 2200
results = DataFrame(damcoef = mappedphi
                  , etscoef = mappedt2
                  , prodcoef = mappedga # This is prod2100 in the xls, but I'm not sure where that is mapped from
                  , carbcoef = mappedmueq
                  , sigcoef = mappedgs
                  , scc2015 = map(r -> r.scc[1],  values(base))
                  , scc2020 = map(r -> r.scc[2],  values(base))
                  , temp2050 = map(r -> r.Tₐₜ[8],  values(base))
                  , temp2100 = map(r -> r.Tₐₜ[18],  values(base))
                  , temp2200 = map(r -> r.Tₐₜ[38],  values(base))
                  , carbonconc2050 = map(r -> r.Mₐₜ[8],  values(base))
                  , carbonconc2100 = map(r -> r.Mₐₜ[18],  values(base))
                  , carbonconc2200 = map(r -> r.Mₐₜ[38],  values(base))
                  , output2050 = map(r -> r.Y[8],  values(base))
                  , output2100 = map(r -> r.Y[18],  values(base))
                  , emis2050 = map(r -> r.Eind[8],  values(base))
                  , emis2100 = map(r -> r.Eind[18],  values(base))
                  , emis2200 = map(r -> r.Eind[38],  values(base))
                  , damfrac2050 = map(r -> r.Ω[8],  values(base))
                  , damfrac2100 = map(r -> r.Ω[18],  values(base))
                  , damfrac2200 = map(r -> r.Ω[38],  values(base))
                  , sigma2050 = map(r -> r.Σ[8],  values(base))
                  , sigma2100 = map(r -> r.Σ[18],  values(base))
                  , sigma2200 = map(r -> r.Σ[38],  values(base))
                  , r2015 = map(r -> r.RI[1],  values(base))
                  , r2100 = map(r -> r.RI[18],  values(base))
                  , Objective = map(r -> r.UTILITY,  values(base))
                  , cca2100 = map(r -> r.CCA[18],  values(base))
                  , cca2200 = map(r -> r.CCA[38],  values(base))
                  , cca2300 = map(r -> r.CCA[58],  values(base))
                  , ppm2050 = map(r -> r.Mₐₜppm[8],  values(base))
                  , ppm2100 = map(r -> r.Mₐₜppm[18],  values(base))
                  , ppm2200 = map(r -> r.Mₐₜppm[38],  values(base))
    );

In [ ]:
results

In [ ]:
colwise(mean, results), colwise(std, results)

In [4]:
mappedgs

5-element Array{Float64,1}:
 -0.02002
 -0.01689
 -0.0152 
 -0.01334
 -0.01055